### Postprocessing Results

In [64]:
import glob as gb
import ntpath # Equivalent of R's base (remove path)
import pandas as pd

In [65]:
# Specify the folder where the results are located
BUCKET = 'bucket19'
#local_dir = f'/Users/jasonpayne/Documents/slu_res/{BUCKET}/'
local_dir = f'C:\\Flexcom_Analysis\\_slug_project\\{BUCKET}\\'
plot_files = gb.glob(f'{local_dir}*.mplt')

In [66]:
# Create a list of load cases that are covered by the timetrace files
lc_files = []
for file_p in plot_files:
    lc_files.append(ntpath.basename(file_p[:len(file_p) - 8]))
    
lc_files = list(dict.fromkeys(lc_files))
lc_files.sort()

In [67]:
# Create empty lists of take processed results
load_cases = []
out_verDisp_mm_n5 = []
out_oopDisp_mm_n5 = []
out_oopDisp_mm_n2 = []
out_endBend_kNm_n6 = []
out_supReac_kN_n3 = []
out_supReac_kN_n4 = []

# Loop through and process each load case and each plot file in each load case
for lc in lc_files:
    load_cases.append(lc)
    for tt in ['T1', 'T3', 'T4', 'T5', 'T6', 'T7']:
        # Form the filename from the load case reference
        file_p = f'{lc}.{tt}.mplt'
        
        # Download blob as StorageStreamDownloader object (stored in memory)
        plot_raw = pd.read_csv(f'{local_dir}{file_p}', skiprows=12, delimiter='"')
        dat_dt = {'time':plot_raw.iloc[:,3].dropna(), 'value':plot_raw.iloc[:,5].dropna()}
        dat_df = pd.DataFrame(dat_dt)

        # Breakdown of types of timetrace
        motions = ['T1', 'T2', 'T3', 'T4']
        moments = ['T5', 'T6']
        reactin = ['T7']

        # Some unit conversions depending on type
        # Use mm for displacement ranges
        if (any(x in file_p for x in motions)):
            dat_df.value = dat_df.value*1000

        elif (any(x in file_p for x in moments)):
            dat_df.value = dat_df.value/1000

        elif (any(x in file_p for x in reactin)):
            dat_df.value = dat_df.value/1000

        # Calculate the range
        dat_range = dat_df.value.max() - dat_df.value.min()
        #print(file_p[:len(file_p) - 8])
        #print(f'Max: {round(dat_df.value.max(), 1)}, Min: {round(dat_df.value.min(), 1)}') 
        #print('Range: ', round(dat_range, 1), '\n')

        # Collect the results
        if 'T1' in file_p:
            out_verDisp_mm_n5.append(round(dat_range, 2))
        elif 'T3' in file_p:
            out_oopDisp_mm_n5.append(round(dat_range, 2))
        elif 'T4' in file_p:
            out_oopDisp_mm_n2.append(round(dat_range, 2))
        elif 'T5' in file_p:
            out_endBend_kNm_n6.append(round(dat_range, 2))
        elif 'T6' in file_p:
            out_supReac_kN_n3.append(round(dat_range, 2)) 
        elif 'T7' in file_p:
            out_supReac_kN_n4.append(round(dat_range, 2))     
    

In [68]:
# Now compile the results in a single dataframe
res_df = pd.DataFrame(load_cases, columns = ['load_case'])
res_df['out_oopDisp_mm_n2'] = out_oopDisp_mm_n2
res_df['out_supReac_kN_n3'] = out_supReac_kN_n3
res_df['out_supReac_kN_n4'] = out_supReac_kN_n4
res_df['out_verDisp_mm_n5'] = out_verDisp_mm_n5
res_df['out_oopDisp_mm_n5'] = out_oopDisp_mm_n5
res_df['out_endBend_kNm_n6'] = out_endBend_kNm_n6
res_df

,load_case,out_oopDisp_mm_n2,out_supReac_kN_n3,out_supReac_kN_n4,out_verDisp_mm_n5,out_oopDisp_mm_n5,out_endBend_kNm_n6
0,18001_sd792_sl9_pd190_pl11_sv5_bp12_ba790_bp57,184.85,14.56,52.75,228.35,174.78,190.60
1,18002_sd834_sl12_pd179_pl13_sv7_bp13_ba680_bp62,201.45,17.57,55.72,260.90,234.86,210.05
2,18003_sd703_sl13_pd188_pl11_sv9_bp12_ba700_bp46,178.32,18.57,60.53,311.82,185.90,264.11
3,18004_sd746_sl9_pd87_pl8_sv9_bp12_ba560_bp56,157.53,13.99,40.48,162.44,155.05,175.54
4,18005_sd742_sl9_pd223_pl11_sv8_bp13_ba480_bp58,140.49,14.15,38.00,178.57,165.11,149.90
...,...,...,...,...,...,...,...
995,18996_sd734_sl10_pd151_pl9_sv4_bp10_ba820_bp61,186.06,15.28,53.23,239.14,162.80,170.49
996,18997_sd681_sl7_pd213_pl11_sv10_bp11_ba830_bp65,218.91,17.62,57.54,267.13,205.14,257.89
997,18998_sd746_sl11_pd147_pl9_sv6_bp14_ba890_bp54,206.83,16.15,57.64,261.26,187.19,220.45
998,18999_sd843_sl13_pd128_pl10_sv11_bp12_ba850_bp48,211.28,19.29,66.92,319.02,250.10,370.68


In [70]:
# Write the results as a csv
res_df.to_csv(f'{BUCKET}_results.csv', index=False)